In [16]:
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=0gZ-l0npPIca

In [1]:
# pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# pip install torch-geometric

In [1]:
import os
import json
import time
import random
#import scipy
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp
from typing import Optional, Tuple
from sklearn.metrics import classification_report

from tqdm import tqdm

import networkx as nx

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# with open("file_info.json", 'r') as f :
#     file_info = json.load(f)
    
names= ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH = '/media/dxlab/storage/junyeop/'
#len(os.listdir(PATH + "image_dep")), len(os.listdir(PATH + "image_daily"))

   

adj_daily = os.listdir(PATH + "adj_mat/daily/")
adj_diagnosis = os.listdir(PATH + "adj_mat/diagnosis/")
adj_before = os.listdir(PATH + "adj_mat/before/")


# daily vlog feature load
with open("visual_features_daily_all.pickle", 'rb') as f :
    visual_features_daily = pickle.load(f)
with open("metadata_features_daily_all.pickle", 'rb') as f :
    meta_daily_features = pickle.load(f)
with open(PATH + 'vlog_metadata_daily.pickle', 'rb') as f :
    meta_daily = pickle.load(f)
with open(PATH + 'vlog_metadata_daily2.pickle', 'rb') as f :
    meta_daily2= pickle.load(f)    
meta_daily.update(meta_daily2)

# depression vlog feature load
with open("visual_features_diagnosis_all.pickle", 'rb') as f :
    visual_features_diagnosis = pickle.load(f)
with open("metadata_features_diagnosis_all.pickle", 'rb') as f :
    meta_diagnosis_features = pickle.load(f) 

# before vlog feature load
with open('metadata_features_before.pickle', 'rb') as f :
    meta_before_features = pickle.load(f)
with open("visual_features_before.pickle", 'rb') as f :
    visual_features_before = pickle.load(f)
    
    
meta_features = meta_daily_features.copy()
print(len(meta_features))
meta_features.update(meta_before_features)
print(len(meta_features))
meta_features.update(meta_diagnosis_features)
print(len(meta_features))

1877
2568
4497


In [3]:
file_info = dict()




have_meta_daily = dict()
for i in adj_daily :
    if os.path.splitext(i)[0] in list(meta_daily.keys()) :
        if type(meta_daily[os.path.splitext(i)[0]]) == str :
            continue
        have_meta_daily[os.path.splitext(i)[0]] = meta_daily[os.path.splitext(i)[0]]
        have_meta_daily[os.path.splitext(i)[0]]['label'] = 0
        try :
            have_meta_daily[os.path.splitext(i)[0]]['visual_feature'] = visual_features_daily[os.path.splitext(i)[0]]
        except :
            del have_meta_daily[os.path.splitext(i)[0]]   

have_meta_diagnosis = dict()
for i in adj_diagnosis :
    if os.path.splitext(i)[0] in list(meta_diagnosis_features.keys()) :
        if type(meta_diagnosis_features[os.path.splitext(i)[0]]) == str :
            continue        
        have_meta_diagnosis[os.path.splitext(i)[0]] = meta_diagnosis_features[os.path.splitext(i)[0]]
        have_meta_diagnosis[os.path.splitext(i)[0]]['label'] = 1
        try :
            have_meta_diagnosis[os.path.splitext(i)[0]]['visual_feature'] = visual_features_diagnosis[os.path.splitext(i)[0]]
        except :
            del have_meta_diagnosis[os.path.splitext(i)[0]]            
            
            
have_meta_before = dict()
for i in adj_before :
    if os.path.splitext(i)[0] in list(meta_before_features.keys()) :
        if type(meta_before_features[os.path.splitext(i)[0]]) == str :
            continue        
        have_meta_before[os.path.splitext(i)[0]] = meta_before_features[os.path.splitext(i)[0]]
        have_meta_before[os.path.splitext(i)[0]]['label'] = 2
        try :
            have_meta_before[os.path.splitext(i)[0]]['visual_feature'] = visual_features_before[os.path.splitext(i)[0]]
        except :
            del have_meta_before[os.path.splitext(i)[0]]
        
        
file_info = dict()
file_info.update(have_meta_daily)
file_info.update(have_meta_diagnosis)
file_info.update(have_meta_before)
        
len(file_info), len(have_meta_daily),  len(have_meta_diagnosis), len(have_meta_before)

(4470, 1877, 2230, 642)

In [4]:
random.seed(0)
file_info = list(file_info.items())
random.shuffle(file_info)
file_info = dict(file_info)

In [5]:
NODE_ATTR = 80
HIDDEN_DIM = 1024
EPOCHS = 100
BATCH_SIZE = 32

In [6]:
feature_matrix = torch.eye(len(names), dtype = torch.float32) # node feature matrix all one vector

In [7]:
labels = list()
c0, c1, c2 = 0, 0, 0

for x in file_info :
    if file_info[x]['label'] == 0:
        labels.append(0)
        c0 += 1
    elif file_info[x]['label'] == 1 :
        labels.append(1)
        c1 += 1
    else :
        labels.append(2)
        c2 += 1 
labels = torch.LongTensor(labels).unsqueeze(dim = 1)
print("daily vlog : {}\ndepression vlog : {}\nbefore vlog : {}".format(c0, c1, c2))

daily vlog : 1877
depression vlog : 1951
before vlog : 642


In [8]:
dataset = list()

for idx, vlog in tqdm(enumerate(file_info)) :

    if labels[idx] == 0 :
        with open(PATH + "adj_mat/daily/" + vlog + '.npy', 'rb') as f:
            m = np.load(f)
    elif labels[idx] == 1:
        with open(PATH + "adj_mat/diagnosis/" + vlog + '.npy', 'rb') as f:
            m = np.load(f)
    else :
        with open(PATH + "adj_mat/before/" + vlog + '.npy', 'rb') as f:
            m = np.load(f)        
    m = nx.adjacency_matrix(nx.from_numpy_array(m))
    edge_index, edge_attr = torch_geometric.utils.from_scipy_sparse_matrix(m)
    data = Data(edge_index=edge_index, x=feature_matrix, edge_attr = edge_attr, y = labels[idx])
    
    if 'description' in file_info[vlog] :
        dataset.append([data, 
                        file_info[vlog]['visual_feature'], 
                        meta_features[vlog]['title'], 
                        meta_features[vlog]['description'], 
                        meta_features[vlog]['duration']])
    
random.shuffle(dataset)

4470it [00:04, 929.72it/s] 


In [9]:
# check duration average
daily_duration, daily_count = 0, 0
depression_duration, depression_count = 0, 0
before_duration, before_count = 0, 0


for x in dataset :
    if x[0].y.item() == 0:
        daily_count += 1
        daily_duration += x[-1].item()
    elif x[0].y.item() == 1:
        depression_count += 1
        depression_duration += x[-1].item()
    else :
        before_count += 1
        before_duration += x[-1].item()
print("daily 평균 길이 : {}\ndepressoin 평균 길이 : {}\n before 평균 길이 : {}".format(daily_duration/daily_count, depression_duration/depression_count, before_duration/before_count))

daily 평균 길이 : 903.3958444326053
depressoin 평균 길이 : 396.9897488467453
 before 평균 길이 : 515.7367601246106


In [10]:
train_dataset = dataset[:int(len(dataset)*0.8)]
valid_dataset = dataset[int(len(dataset)*0.8):int(len(dataset)*0.9)]
test_dataset = dataset[int(len(dataset)*0.9):]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
y_train = [x[0].y for x in train_dataset]
y_valid = [x[0].y for x in valid_dataset]
y_test = [x[0].y for x in test_dataset]

In [16]:
y_test.count(0), y_test.count(1), y_test.count(2)

(191, 192, 64)

In [75]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int = HIDDEN_DIM, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

    def forward(self,
                query: Tensor,
                key: Tensor,
                value: Tensor,
                mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)      # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)  # BxNxQ_LENxK_LEN

        context, attn = self.scaled_dot_attn(query, key, value, mask)

        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn

In [76]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(NODE_ATTR, HIDDEN_DIM)
        self.conv2 = GATConv(HIDDEN_DIM, HIDDEN_DIM)
        self.conv3 = GATConv(HIDDEN_DIM, HIDDEN_DIM)        
        
        self.self_attn = MultiHeadAttention()
        self.cross_attn = MultiHeadAttention()
    
        self.visual_linear = nn.Linear(1000, HIDDEN_DIM)   
    
        # reduce dim
        self.title_linear = nn.Linear(768, HIDDEN_DIM//2)
        self.description_linear = nn.Linear(768, HIDDEN_DIM//2)
        
        self.dim_reduce = nn.Linear(2 * HIDDEN_DIM + 1, HIDDEN_DIM)
        
        self.lin1 = Linear(HIDDEN_DIM, HIDDEN_DIM//2)
        self.lin2 = Linear(HIDDEN_DIM//2, HIDDEN_DIM//4)
        
        self.classifier = Linear(HIDDEN_DIM//4, len(labels.unique()))
            
    def forward(self, x, edge_index, batch, visual_feature, title, description, duration):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = self.self_attn(x, x, x)[0]     
        
        x_visual = self.visual_linear(visual_feature).squeeze(dim = 1)
        
        description = self.description_linear(description).squeeze(dim = 1)
        title = self.title_linear(title).squeeze(dim = 1)
        duration = duration
        #print(x_visual.shape, description.shape, title.shape, duration.shape)
        x_all = torch.cat([x_visual, description, title, duration], dim = 1)
        x_all = self.dim_reduce(x_all)
        
        x = self.cross_attn(x, x_all, x_all)[0]
        
        x = F.dropout(self.lin1(x), p=0.5, training=self.training)
        x = F.dropout(self.lin2(x), p=0.5, training=self.training)
        
        x = self.classifier(x)
        return x.squeeze(dim = 1)

In [77]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Model().to(device)

MODEL_PATH = './checkpoint/grid_search/3_multiclass_multimodal/GAT/'

In [78]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-3)
criterion = torch.nn.CrossEntropyLoss()

def train(opt):
    model.train()
    correct = 0
    total_loss = 0.0
    for data, visual_feature, title, description, duration in train_loader: # Iterate in batches over the training dataset.
        
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)

        visual_feature = visual_feature.to(device)
        title = title.to(device)
        description = description.to(device)
        duration = duration.to(device)

#        description = bert_encoder(description).pooler_output
#        title = bert_encoder(title).pooler_output   

        out = model(x = data.x, 
                    edge_index = data.edge_index, 
                    batch = data.batch, 
                    visual_feature = visual_feature,
                    title = title, 
                    description = description, 
                    duration = duration)  # Perform a single forward pass.

        loss = criterion(out, data.y).to(device)  # Compute the loss.

        loss.backward()  # Derive gradients.
        opt.step()  # Update parameters based on gradients.
        opt.zero_grad()  # Clear gradients.
        pred = out.argmax(dim = 1)
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.        
        total_loss += loss.item()
        
    return correct / len(train_loader.dataset), total_loss / len(train_loader)

def test(loader):
    model.eval()

    correct = 0
    total_loss = 0.0  
    for data, visual_feature, title, description, duration in loader:  # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        
        title = title.to(device)
        description = description.to(device)
        duration = duration.to(device)
        visual_feature = visual_feature.to(device)
#         description = bert_encoder(description).pooler_output
#         title = bert_encoder(title).pooler_output   
        
        out = model(data.x, data.edge_index, data.batch, visual_feature, title, description, duration)
        loss = criterion(out, data.y).to(device)  # Compute the loss.
        pred = out.argmax(dim = 1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        
        total_loss += loss.item()
        
    return correct/len(loader.dataset), total_loss / len(loader) # Derive ratio of correct predictions.


In [79]:
LR_LIST = [1e-4, 1e-5, 1e-6]
WEIGHT_DECAY = [1e-3, 1e-4, 1e-5]

grid_result = pd.DataFrame(columns=['epoch', 'learning_rate', 'weight_decay', 
                                    'train_acc', 'valid_acc', 
                                    'train_loss', 'valid_loss'],
                           index= range(len(LR_LIST) * len(WEIGHT_DECAY)))

iter_ = 0
patience = 0

BEST_VALID_ACC = 0
BEST_LR = 0
BEST_WD = 0
prev_valid_loss = 1e+10

for lr in LR_LIST :
    for wd in WEIGHT_DECAY :
        model = Model().to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        criterion = torch.nn.CrossEntropyLoss()

        train_loss_list = list()
        
        valid_loss_list = list()

        min_valid_loss = 2e+10

        best_epoch = 0
        pbar = tqdm(range(EPOCHS), 
                    ascii = ' =')
        
        for epoch in pbar :
        #for epoch in tqdm(range(EPOCHS)) :
            
            time.sleep(1)
            train_acc, train_loss = train(optimizer)
            valid_acc, valid_loss = test(valid_loader)
            train_loss_list.append(train_loss)
            valid_loss_list.append(valid_loss)
            
            if valid_loss < min_valid_loss :
                best_epoch = epoch
                torch.save(model, MODEL_PATH + "best_model_binary_{}_{}.pt".format(lr, wd))
                torch.save(model.state_dict(), MODEL_PATH + 'best_model_binary_parameters_{}_{}.pt'.format(lr, wd))
                min_valid_loss = valid_loss
                grid_result.iloc[iter_, :] = [epoch, lr, wd, 
                                              train_acc, train_loss, 
                                              valid_acc, valid_loss]
                best_ta = train_acc
                best_va = valid_acc
                best_tl = train_loss
                best_vl = valid_loss
                
                
            else :
                patience += 1        
            pbar.set_description(f'Epoch {epoch+1} ')
            pbar.set_postfix({'LR' : lr,
                              'WD' : wd,
                              'TA' : train_acc,
                              'TL' : train_loss, 
                              'VA' : valid_acc,
                              'VL' : valid_loss})        
        print("Epoch : {}, LR : {}, WD : {}, Train Acc : {}, Valid Acc : {} , Train Loss : {}, Valid Loss : {}".format(epoch +1, lr, wd, 
                                                                                                                       best_ta, best_va, 
                                                                                                                       best_tl, best_vl))
        if prev_valid_loss > best_vl :
            BEST_VALID_ACC = best_va
            BEST_LR = lr
            BEST_WD = wd
            prev_valid_loss = best_vl
        

Epoch 100 : 100%|=======================================================================================| 100/100 [04:45<00:00,  2.86s/it, LR=0.0001, WD=0.001, TA=0.708, TL=0.748, VA=0.729, VL=0.74]


Epoch : 100, LR : 0.0001, WD : 0.001, Train Acc : 0.7044183445190156, Valid Acc : 0.7427293064876958 , Train Loss : 0.7685129182147128, Valid Loss : 0.667555034160614


Epoch 100 : 100%|=====================================================================================| 100/100 [04:38<00:00,  2.79s/it, LR=0.0001, WD=0.0001, TA=0.764, TL=0.631, VA=0.734, VL=0.675]


Epoch : 100, LR : 0.0001, WD : 0.0001, Train Acc : 0.7569910514541387, Valid Acc : 0.7829977628635347 , Train Loss : 0.6374494984213795, Valid Loss : 0.596164869410651


Epoch 100 : 100%|=======================================================================================| 100/100 [05:09<00:00,  3.10s/it, LR=0.0001, WD=1e-5, TA=0.784, TL=0.565, VA=0.747, VL=0.683]


...

Epoch : 100, LR : 1e-06, WD : 0.001, Train Acc : 0.6420581655480985, Valid Acc : 0.7002237136465325 , Train Loss : 0.9567862706524985, Valid Loss : 0.7827978985650199


Epoch 100 : 100%|=======================================================================================| 100/100 [05:34<00:00,  3.34s/it, LR=1e-6, WD=0.0001, TA=0.661, TL=0.936, VA=0.696, VL=0.807]


Epoch : 100, LR : 1e-06, WD : 0.0001, Train Acc : 0.6442953020134228, Valid Acc : 0.7046979865771812 , Train Loss : 0.9526668576789754, Valid Loss : 0.7790958625929696


Epoch 100 : 100%|==========================================================================================| 100/100 [05:35<00:00,  3.35s/it, LR=1e-6, WD=1e-5, TA=0.66, TL=0.936, VA=0.696, VL=0.807]

Epoch : 100, LR : 1e-06, WD : 1e-05, Train Acc : 0.6445749440715883, Valid Acc : 0.7046979865771812 , Train Loss : 0.9524617030152253, Valid Loss : 0.7789109306676048


In [80]:
LR_LIST = [1e-4, 1e-5, 1e-6]
WEIGHT_DECAY = [1e-3, 1e-4, 1e-5]

def inference(loader, lr, wd):
    y_pred = list()
    y_test = list()
    
    best_model = torch.load(MODEL_PATH + "best_model_binary_{}_{}.pt".format(lr, wd))
    best_model.eval()

    correct = 0
    for data, visual_feature, title, description, duration in loader: # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        visual_feature = visual_feature.to(device)
        title = title.to(device)
        description = description.to(device)
        duration = duration.to(device)
        
#        description = bert_encoder(description).pooler_output
#        title = bert_encoder(title).pooler_output   
        
        out = best_model(data.x, data.edge_index, data.batch, visual_feature, title, description, duration)        

        pred = out.argmax(dim=1)  # Use the class with highest probability.
        
        y_pred.extend(pred.tolist())
        y_test.extend(data.y.tolist())
        
    return y_test, y_pred

In [82]:
target_names = ['daily', "depression",'before']

In [83]:
#epoch 500
for lr in LR_LIST :
    for wd in WEIGHT_DECAY :
        print(lr, wd)
        y_test, y_pred = inference(test_loader, lr, wd)
        print(classification_report(y_test, y_pred, target_names = target_names, digits = 4))

0.0001 0.001
              precision    recall  f1-score   support

       daily     0.7458    0.6911    0.7174       191
  depression     0.6967    0.7656    0.7295       192
      before     0.6780    0.6250    0.6504        64

    accuracy                         0.7136       447
   macro avg     0.7068    0.6939    0.6991       447
weighted avg     0.7150    0.7136    0.7130       447

0.0001 0.0001
              precision    recall  f1-score   support

       daily     0.7574    0.8010    0.7786       191
  depression     0.7241    0.7656    0.7443       192
      before     0.9524    0.6250    0.7547        64

    accuracy                         0.7606       447
   macro avg     0.8113    0.7306    0.7592       447
weighted avg     0.7710    0.7606    0.7605       447

0.0001 1e-05
              precision    recall  f1-score   support

       daily     0.8315    0.7749    0.8022       191
  depression     0.7225    0.8542    0.7828       192
      before     0.9286    0.6094  

In [84]:
y_test, y_pred = inference(test_loader, BEST_LR, BEST_WD)
print(classification_report(y_test, y_pred, target_names = target_names, digits=4))

              precision    recall  f1-score   support

       daily     0.8315    0.7749    0.8022       191
  depression     0.7225    0.8542    0.7828       192
      before     0.9286    0.6094    0.7358        64

    accuracy                         0.7852       447
   macro avg     0.8275    0.7461    0.7736       447
weighted avg     0.7985    0.7852    0.7844       447



In [85]:
BEST_LR, BEST_WD

(0.0001, 1e-05)